In [1]:
import urllib, os, sys, zipfile
from os.path import dirname
import numpy as np
import tensorflow as tf
from tensorflow.core.framework import graph_pb2
from tensorflow.python.tools.freeze_graph import freeze_graph
import tfcoreml
import inference_wrapper
import configuration
from coremltools.proto import NeuralNetwork_pb2
from PIL import Image
%pdb on

Automatic pdb calling has been turned ON


In [2]:
checkpoint_file = './trainlogIncNEW/model.ckpt-1000000'
pre_frozen_model_file = './frozen_model_textgenNEW.pb'
frozen_model_file = './frozen_model_textgenNEW.pb'
#output_node_names = ['lstm/initial_state']
output_node_names = ['softmax','lstm/state',
'lstm/split',
'lstm/basic_lstm_cell/concat_1',
'lstm/basic_lstm_cell/split_1',
'lstm/basic_lstm_cell/einsum_1/MatMul',
'lstm/basic_lstm_cell/einsum/MatMul',
'lstm/basic_lstm_cell/concat_2',]
#'lstm/basic_lstm_cell/einsum/MatMul',
#'lstm/basic_lstm_cell/einsum/Reshape',
#'lstm__basic_lstm_cell__einsum_1__Reshape__0',
#'lstm/basic_lstm_cell/einsum_1/MatMul',
#'lstm__basic_lstm_cell__einsum__Reshape_1__0',
# 'lstm__basic_lstm_cell__einsum_1__Reshape_1__0',                    
#'lstm/basic_lstm_cell/concat_2']
# output_node_names = ['lstm/split']
# original_output_node_names = ['softmax','lstm/state']
# input_node_names = ['image_feed', 'seq_embeddings','lstm/state_feed']
#input_node_names = ['image_feed']
input_node_names = ['seq_embeddings','lstm/state_feed']
beam_size = 2

In [3]:
# Build the inference graph.
g = tf.Graph()
with g.as_default():
    model = inference_wrapper.InferenceWrapper()
    restore_fn = model.build_graph_from_config(configuration.ModelConfig(),
                                               checkpoint_file)
g.finalize()

INFO:tensorflow:Building model.
About to decide if splitting
new_h Tensor("lstm/basic_lstm_cell/Mul_2:0", shape=(1, 512), dtype=float32)
new_state LSTMStateTuple(c=<tf.Tensor 'lstm/basic_lstm_cell/Add_1:0' shape=(1, 512) dtype=float32>, h=<tf.Tensor 'lstm/basic_lstm_cell/Mul_2:0' shape=(1, 512) dtype=float32>)
About to decide if splitting
It's splitting
h Tensor("lstm/Squeeze_1:0", shape=(1, 2, 512), dtype=float32)
inputs Tensor("seq_embeddings:0", shape=(1, 2, 300), dtype=float32)
gate_inputs0 Tensor("lstm/basic_lstm_cell/einsum/Reshape_1:0", shape=(1, 1, 2048), dtype=float32)
gate_inputs1 Tensor("lstm/basic_lstm_cell/einsum_1/Reshape_1:0", shape=(1, 1, 2048), dtype=float32)
squeeze Tensor("lstm/basic_lstm_cell/Squeeze_2:0", shape=(812,), dtype=float32)
<tf.Variable 'lstm/basic_lstm_cell/kernel:0' shape=(812, 2048) dtype=float32_ref>
gate_inputs Tensor("lstm/basic_lstm_cell/concat_2:0", shape=(1, 2, 2048), dtype=float32)
new_h Tensor("lstm/basic_lstm_cell/Mul_5:0", shape=(1, 2, 512), 

In [4]:
# Write the graph
tf_model_path = './log/pre_graph_textgenNEW.pb'
tf.train.write_graph(
    g,
    './log',
    'pre_graph_textgenNEW.pb',
    as_text=False,
)

with open(tf_model_path, 'rb') as f:
    serialized = f.read()
tf.reset_default_graph()
original_gdef = tf.GraphDef()
original_gdef.ParseFromString(serialized)

47084205

In [5]:
from tensorflow.python.tools import strip_unused_lib
from tensorflow.python.framework import dtypes
from tensorflow.python.platform import gfile

gdef = strip_unused_lib.strip_unused(
        input_graph_def = original_gdef,
        input_node_names = input_node_names,
        output_node_names = output_node_names,
        placeholder_type_enum = dtypes.float32.as_datatype_enum)
# Save it to an output file
with gfile.GFile(pre_frozen_model_file, 'wb') as f:
    f.write(gdef.SerializeToString())

In [6]:
# Call freeze graph
freeze_graph(input_graph=pre_frozen_model_file,
             input_saver='',
             input_binary=True,
             input_checkpoint=checkpoint_file,
             output_node_names=','.join(output_node_names),
             restore_op_name='save/restore_all',
             filename_tensor_name='save/Const:0',
             output_graph=frozen_model_file,
             clear_devices=True,
             initializer_nodes='')

INFO:tensorflow:Restoring parameters from ./trainlogIncNEW/model.ckpt-1000000
INFO:tensorflow:Froze 4 variables.
Converted 4 variables to const ops.


In [7]:
input_tensor_shapes = {
     #'image_feed:0': [299, 299, 3],
    'seq_embeddings:0': [1, beam_size, 300],
    'lstm/state_feed:0': [1, beam_size, 1024],
}
coreml_model_file = './Textgen_NEW.mlmodel'
# output_tensor_names = ['softmax:0','lstm/state:0','lstm/initial_state:0']
# output_tensor_names = ['lstm/initial_state:0']
output_tensor_names = [[node + ':0'] if not 'split' in node else [node + ':0',node + ':1'] for node in output_node_names]
output_tensor_names = [item for sublist in output_tensor_names for item in sublist]
print(output_tensor_names)

['softmax:0', 'lstm/state:0', 'lstm/split:0', 'lstm/split:1', 'lstm/basic_lstm_cell/concat_1:0', 'lstm/basic_lstm_cell/split_1:0', 'lstm/basic_lstm_cell/split_1:1', 'lstm/basic_lstm_cell/einsum_1/MatMul:0', 'lstm/basic_lstm_cell/einsum/MatMul:0', 'lstm/basic_lstm_cell/concat_2:0']


In [8]:
# Call the converter
#output_feature_names = ['lstm/basic_lstm_cell/concat_2:0']

coreml_model = tfcoreml.convert(
        tf_model_path=frozen_model_file, 
        mlmodel_path=coreml_model_file, 
        input_name_shape_dict=input_tensor_shapes,
#         output_feature_names=output_feature_names + ['lstm/basic_lstm_cell/MatMul_1:0'],
        output_feature_names=output_tensor_names,
        #image_input_names = ['image_feed:0'],
         #red_bias = -1,
         #green_bias = -1,
         #blue_bias = -1,
         #image_scale = 2.0/255.0
#         add_custom_layers=True,
#         custom_conversion_functions={
#             'lstm/split': _convert_split,
#             'lstm/basic_lstm_cell/split_1': _convert_lstmsplit,
#             'lstm/basic_lstm_cell/split': _convert_lstmsplit,
#         },
)

Shapes not found for 23 tensors. Executing graph to determine shapes. 
Automatic shape interpretation succeeded for input blob seq_embeddings:0
Automatic shape interpretation succeeded for input blob lstm/state_feed:0
1/85: Analysing op name: seq_embeddings ( type:  Placeholder )
Skipping name of placeholder
2/85: Analysing op name: lstm/basic_lstm_cell/kernel ( type:  Const )
3/85: Analysing op name: lstm/basic_lstm_cell/kernel/read ( type:  Identity )
4/85: Analysing op name: lstm/basic_lstm_cell/bias ( type:  Const )
5/85: Analysing op name: lstm/basic_lstm_cell/bias/read ( type:  Identity )
6/85: Analysing op name: lstm/state_feed ( type:  Placeholder )
Skipping name of placeholder
7/85: Analysing op name: lstm/ExpandDims/dim ( type:  Const )
8/85: Analysing op name: lstm/ExpandDims ( type:  ExpandDims )
9/85: Analysing op name: lstm/split/split_dim ( type:  Const )
10/85: Analysing op name: lstm/split ( type:  Split )
11/85: Analysing op name: lstm/Squeeze ( type:  Squeeze )
12/85

AssertionError: Split op case not handled. Input shape = [1, 1, 2, 812], output shape = [1, 1, 1, 812]

> /Users/ALP/PycharmProjects/MemeProject/venv3/lib/python2.7/site-packages/tfcoreml/_shape_sensitive_layers.py(217)_add_split()
    215          len(common_out_shape) == 4 and \
    216          common_out_shape[:3] == input_shape[:3], \
--> 217         ('Split op case not handled. Input shape = {}, output shape = {}'.format(str(input_shape), str(common_out_shape)))
    218 
    219   context.builder.add_split(output_names[0], input_name, output_names)


KeyboardInterrupt


In [ ]:
Image.fromarray(np.uint8(np.ones([299,299,3])*100))

In [ ]:
np.random.seed(2)
y = np.array([[[x,x]] for x in np.random.rand(300)])
print(y[10,0,0])
print(y[10,0,1])

In [ ]:
np.random.seed(2)
seq_embeddings = np.ones([300,1,2])*0.5 #np.array([[[x,x]] for x in np.ones(300)*0.5])
state_feed = np.ones([1024,1,2])*0.6 #np.array([[[x,x]] for x in np.ones(1024)*0.6])

#image_feed = Image.fromarray(np.uint8(np.ones([299,299,3])*100)) #np.ones([3,299,299])
coreml_inputs = {
    'seq_embeddings__0': seq_embeddings,
    'lstm__state_feed__0': state_feed,
    #'image_feed__0': image_feed,
}
coreml_output = coreml_model.predict(coreml_inputs, useCPUOnly=True)
# print(coreml_output['lstm__state__0'].shape)
# print(coreml_output['softmax__0'].shape)
# print(coreml_output['softmax__0'].reshape(38521, 1, 2))
# print(coreml_output)
print(coreml_output['lstm__state__0'].shape)
print(coreml_output['softmax__0'].shape)
print(coreml_output['lstm__basic_lstm_cell__split_1__1'].shape)
print(coreml_output['lstm__split__1'].shape)
#print(coreml_output['lstm__initial_state__0'].shape)

In [ ]:
print(coreml_output['lstm__basic_lstm_cell__split_1__0'][695:705,0,0])
print(coreml_output['lstm__basic_lstm_cell__split_1__1'][695:705,0,0])
print(coreml_output['lstm__basic_lstm_cell__split_1__1'].shape)

In [ ]:
print(coreml_output['lstm__split__1'][0,0,-10:-1,0,0])
print(coreml_output['lstm__split__0'][0,0,-10:-1,0,0])

In [ ]:
print(coreml_output['lstm__basic_lstm_cell__concat_1__0'].shape)
print(coreml_output['lstm__basic_lstm_cell__concat_1__0'][0,0,295:305,0,0])
print(coreml_output['lstm__basic_lstm_cell__concat_1__0'][0,0,295:305,0,1])

In [ ]:
print(coreml_output['lstm__basic_lstm_cell__split_1__0'][:10,0,0])
print(coreml_output['lstm__basic_lstm_cell__split_1__1'][:10,0,0])

In [ ]:
print(coreml_output['lstm__basic_lstm_cell__concat_2__0'][0,0,295:305,0,0])
print(coreml_output['lstm__basic_lstm_cell__concat_2__0'][0,0,295:305,0,1])

In [ ]:
print(coreml_output['lstm__state__0'][0,0,:10,0,0])
print(coreml_output['lstm__state__0'][0,0,:10,0,1])

In [ ]:
print(coreml_output['lstm__initial_state__0'][0])
print(coreml_output['lstm__initial_state__0'][-1])

In [ ]:
print(coreml_output['lstm__state__0'][:,:,:10,:,0])

In [ ]:
state_output = np.loadtxt('stateoutputTEST.txt')

In [ ]:
(state_output[1,:]==coreml_output['lstm__state__0'][0,0,:,0,1])[:50]

In [ ]:
print(state_output[0,377])
print(coreml_output['lstm__state__0'][0,0,377,0,0])

In [ ]:
img = np.moveaxis(np.asarray(image) / 256.0, [0, 1, 2], [1, 2, 0])

In [ ]:
coreml_model

In [ ]:
""" 'image_feed:0': [299, 299, 3],
    'seq_embeddings:0': [1, 1, 300],
    'lstm/state_feed:0': [1, 1024],
"""
coreml_inputs = {
    'image_feed__0': image,
#     'seq_embeddings__0': np.random.rand(300, 1, 1),
#     'lstm__state_feed__0': np.random.rand(1024),
}
coreml_output = coreml_model.predict(coreml_inputs, useCPUOnly=True)
print(list(coreml_output['lstm__initial_state__0'][:20]))

In [ ]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1000px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [ ]:
show_graph(gdef)

In [ ]:
embeddings_bytes = embeddings.flatten().tobytes()

In [ ]:
open('embeddings_bin.bin', 'wb').write(embeddings_bytes)

In [ ]:
import coremltools
read_model = coremltools.models.MLModel(coreml_model_file)
dir(read_model)

In [ ]:
import coremltools
def print_coreml_nn_layer_info(spec):
    nn_layers = coremltools.models.utils._get_nn_layers(spec)
    for i, layer in enumerate(nn_layers):
        if layer.WhichOneof('layer') == 'custom':
            print 'layer_id = ', i
            print layer
        else:
            print('{}: layer type: ({}) , inputs: {}, outputs: {}'.
              format(i,layer.WhichOneof('layer'), ", ".join([x for x in layer.input]), ", ".join([x for x in layer.output])))

print_coreml_nn_layer_info(coreml_model.get_spec())

In [ ]:
# Test the model!!!

import PIL
from IPython.core.display import Image

filename = '/Users/freedmand/Desktop/zelda_driving.jpg'
with tf.gfile.GFile(filename, "rb") as f:
  image = PIL.Image.open(f)
  image = image.resize([299,299], PIL.Image.ANTIALIAS)
    
display(Image(filename, width=299, height=299))